## 事前準備

In [1]:
# google driveへの接続

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## CONST
定数の宣言  
ここは各自の環境に合わせて変更してください。ここではGoogleDriveを用いて行うことにします。  
ルートフォルダ内にdataフォルダを作成し、そこにダウンロードしたzipファイルを入れていください。

In [3]:
## rootフォルダ
ROOT_DIR = "/content/drive/MyDrive/Colab Notebooks/unpack-etlcdb"

## 出力フォルダ
OUTPUT_DIR = ROOT_DIR+ "/output"

## DATA
DATE_DIR = "/data"

In [24]:
### フォルダ作成
import os
#### 上記のフォルダがなかった場合、作成する処理をここに入れる

## 処理

### import

In [6]:
### import
import shutil

import struct
from PIL import Image, ImageEnhance

import glob
import os

### unzip

In [ ]:
etl9b_path = '/content/drive/MyDrive/code/ETLCDB/ETL9B.zip' # ここにetl9bのパスを入力

shutil.unpack_archive(etl9b_path ,OUTPUT_DIR+"/unzip")

### binary -> png

In [ ]:

dataset_dir = '/content/drive/MyDrive/Colab Notebooks/unpack-etlcdb/output/unzip'

files = glob.glob(dataset_dir + '/ETL9B/*') # 解凍したバイナリファイルのpath

for fname in files:
    if fname == dataset_dir + "/ETL9B/ETL9BINFO": continue # 情報ファイルは飛ばす
    if fname == dataset_dir + "/ETL9B/output": continue
    print(fname)

    f = open(fname, 'rb')
    # f.seek(0) # ポインタを先頭に
    f.seek(577)
    while True:

        s = f.read(576) 
        if not s: break
        # バイナリデータなのでPythonが理解できるように抽出
        r = struct.unpack('>2H4s504s64x', s)

        kanjicode_jis = r[1]
        #test
        print(kanjicode_jis)
        print(r)
        #/test

        # 画像を抽出
        iF = Image.frombytes('1', (64, 63), r[3], 'raw')
        iP = iF.convert('L')
        dir = OUTPUT_DIR+ "/etl9b/" + str(kanjicode_jis)
        print(dir)
        if not os.path.exists(dir): 
          os.makedirs(dir)
        fn = "{0:02x}-{1:02x}{2:04}.png".format(kanjicode_jis, r[0], r[2])
        # fn = "{0:02x}-{1:02x}.png".format(kanjicode_jis, r[0])
        fullpath = dir + "/" + fn
        if os.path.exists(fullpath): continue
        enhancer = ImageEnhance.Brightness(iP)
        iE = enhancer.enhance(16)
        iE.save(fullpath, 'PNG')

### make-pickle

### learning